# Bibliotecas

In [ ]:
import math
import os
import re
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from collections import Counter
from datetime import timedelta
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support , roc_auc_score, auc, precision_score
from timeit import default_timer as timer

%run ./base_functions.ipynb

pd.options.mode.chained_assignment = None # desabilita warnings de SettingWithCopyWarning
plt.rcParams.update({'figure.max_open_warning': 0}) # desabilita warning para plot de mais de 20 figuras

In [ ]:
# Caminho no qual as imagens geradas serão salvas quando exportadas
saving_path = r"C:\Users\anaso\Desktop\workspace\doutorado\imagens"

# Funções

In [ ]:
class PCA():
   
    # ---------------------------------------------------------------------------
    # Método construtor
    # Função que será chamada toda vez que um objeto PCA for inicializado
    # O modelo selecionará quantos PCs forem necessários para cumprir o % de variabilidade explicada especificado em a

    def __init__ (self, a=0.9):

        # se 0<=a<1,  'a' indica a fraçao de variancia explicada desejada
        # se a>=1,    'a' indica o numero de componentes desejado
        self.a = a
   
    # ---------------------------------------------------------------------------
    # Função para treino do modelo
    # X são os dados de treino
    
    def fit(self, X, conf_Q=0.99, conf_T2=0.99, plot=True):
    
        # guardando médias e desvios-padrão do treino (de cada coluna)
        self.mu_train = X.mean(axis=0)
        self.std_train = X.std(axis=0)        
       
        # normalizando dados de treino
        X = np.array(((X - self.mu_train)/self.std_train))
       
        # calculando a matriz de covariâncias dos dados
        Cx = np.cov(X, rowvar=False)
        
        # aplicando decomposição em autovalores e autovetores
        self.L, self.P = np.linalg.eig(Cx)
        
        # frações da variância explicada
        fv = self.L/np.sum(self.L)
        
        # frações da variância explicada acumuladas
        fva = np.cumsum(self.L)/sum(self.L)
       
        # definindo número de componentes
        if (self.a>0 and self.a<1):
            self.a = np.where(fva>self.a)[0][0]+1 
            
        # calculando limites de detecção

        # limite da estatística T^2
        from scipy.stats import f
        F = f.ppf(conf_T2, self.a, X.shape[0]-self.a)
        self.T2_lim = ((self.a*(X.shape[0]**2-1))/(X.shape[0]*(X.shape[0]-self.a)))*F
        print("T2_lim: ", self.T2_lim)
        
        # limite da estatística Q
        theta = [np.sum(self.L[self.a:]**(i)) for i in (1,2,3)]
        ho = 1-((2*theta[0]*theta[2])/(3*(theta[1]**2)))
        from scipy.stats import norm
        nalpha = norm.ppf(conf_Q)
        self.Q_lim = (theta[0]*(((nalpha*np.sqrt(2*theta[1]*ho**2))/theta[0])+1+
                                ((theta[1]*ho*(ho-1))/theta[0]**2))**(1/ho))
        print("Q_lim: ", self.Q_lim)
        
        # plotando variâncias explicadas
        if plot:
            fig, ax = plt.subplots()
            ax.bar(np.arange(len(fv)),fv)
            ax.plot(np.arange(len(fv)),fva)
            ax.set_xlabel('Número de componentes')
            ax.set_ylabel('Variância dos dados')
            ax.set_title('PCA - Variância Explicada');

    # ---------------------------------------------------------------------------
    # Função para teste do modelo
            
    def predict(self, X):
            
        # normalizando dados de teste (usando os parâmetros do treino!)
        X = np.array((X - self.mu_train)/self.std_train)

        # calculando estatística T^2
        T = X@self.P[:,:self.a]
        self.T2 = np.array([T[i,:]@np.linalg.inv(np.diag(self.L[:self.a]))@T[i,:].T for i in range(X.shape[0])])
        
        # calculando estatística Q
        e = X - X@self.P[:,:self.a]@self.P[:,:self.a].T
        self.Q  = np.array([e[i,:]@e[i,:].T for i in range(X.shape[0])])
        
        # calculando contribuições para Q
        self.c = np.absolute(X*e) 
                
    # ---------------------------------------------------------------------------
    # Função para plotar cartas de controle
    
    def plot_control_charts(self, fault = None):
     
        fig, ax = plt.subplots(1,2, figsize=(15,3))
        
        ax[0].semilogy(self.T2,'.')
        ax[0].axhline(self.T2_lim,ls='--',c='r');
        ax[0].set_title('Carta de Controle $T^2$')
        
        ax[1].semilogy(self.Q,'.')
        ax[1].axhline(self.Q_lim,ls='--',c='r')
        ax[1].set_title('Carta de Controle Q')
 
        if fault is not None:
            ax[0].axvline(fault, c='w')
            ax[1].axvline(fault, c='w')

    # ---------------------------------------------------------------------------
    # Função para plotar mapas de contribuição
            
    def plot_contributions(self, fault=None, index=None, columns=None):

        fig, ax = plt.subplots(figsize=(20, 6))
        
        c = pd.DataFrame(self.c, index=index, columns=columns)
    
        sns.heatmap(c, ax=ax, yticklabels=int(self.c.shape[0]/10), cmap = plt.cm.Blues)
        
        ax.set_title('Contribuições parciais para Q')
        
        if fault is not None:
            ax.axhline(y=c.index[fault], ls='--', c='k')

In [ ]:
def print_overview(data):
    
    """
    Retorna algumas informações gerais sobre o banco de dados, como existência de nulos e duplicatas, dentre outros.
    """
    
    # Shape do banco
    print("Shape do banco: ", data.shape)
    
    # Alguma linha com todos os valores zerados?
    zero_rows = data.index[(data==0).all(axis=1)].tolist()
    print("\nQuantidade de linhas zeradas: ", len(zero_rows))
    
    # Existem nulos?
    print("Quantidade de nulos no banco: ", data.isnull().sum().sum())
    
    # Há duplicatas baseado no timestamp?
    qtd_duplis = len(data[data.duplicated(['timestamp'])])
    if (qtd_duplis > 0):
        print("Quantidade de duplicatas: ", qtd_duplis)
#         print(data[data.duplicated(['timestamp'])])
    else:
        print("\nNão há registros duplicados.")
    
    print()
    
    # Alguma coluna possui um único valor constante?
    single_values_columns = []
    for col in (data.columns):
        unique_values = data[col].nunique()
        if (unique_values == 1):
            single_values_columns.append(col)
    if len(single_values_columns) > 0:
        print("Existem " + str(len(single_values_columns)) + " colunas constantes. São elas: " + str(single_values_columns))
    else:
        print("Não há colunas constantes.")
    
    # Alguma linha inteiramente zerada?
    data_sem_dt = data.drop(['timestamp'], 1)
    zero_rows = data_sem_dt.index[(data_sem_dt == 0).all(axis=1)].tolist()
    if len(zero_rows) > 0:
        print("Existem " + str(len(zero_rows)) + " linhas inteiramente zeradas.")
    else:
        print("Não há linhas inteiramente zeradas.")
    
    # Alguma linha com todas as colunas NaN?
    nan_rows = data.index[(data == float('nan')).all(axis=1)].tolist()
    if len(nan_rows) > 0:
        print("Existem " + str(len(nan_rows)) + " instâncias sem registro.")
    else:
        print("Não há linhas inteiramente sem registro (nan).")

# Leitura dos arquivos

In [ ]:
data = pd.read_csv(r"D:\Caso 2 - Bomba Centrífuga\final_data.csv", sep=';')
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [ ]:
fault_log = pd.read_excel("D:\Caso 2 - Bomba Centrífuga\Bancos tratados\Falhas_Bomba_final_v0.xlsx")

fault_log['InicioAvar'] = pd.to_datetime(fault_log['InicioAvar'])
fault_log['Dt.real fim'] = pd.to_datetime(fault_log['Dt.real fim'])

# Remoção das falhas não marcadas - longas ou contidas em outras falhas
fault_log = fault_log.dropna(subset=['Filtro'])

In [ ]:
# Informações gerais sobre o banco

print_overview(data)

# Banco 1 - Rótulos via OMs

In [ ]:
labeled_data = data.copy()

labeled_data['rotulos_multi'] = np.zeros(())
labeled_data['rotulos_bin'] = np.zeros(())

In [ ]:
for i in range(len(fault_log)):
    
    ID = fault_log.iloc[i]['ID']
    
    print("\n--------------------------------------------------------------------------------")
    print("Falha ", (ID))

    init_falha = fault_log.iloc[i]['InicioAvar']
    init_falha = datetime.datetime.combine(init_falha, datetime.time.min)

    end_falha = fault_log.iloc[i]['Dt.real fim']
    end_falha = datetime.datetime.combine(end_falha, datetime.time.max)

    df_fault_final = labeled_data.loc[((labeled_data["timestamp"] >= init_falha) & 
                                       (labeled_data["timestamp"] <= end_falha))]
    elapsed_time = end_falha - init_falha

    print("Dias:  ", elapsed_time.days)
    print("Horas: ", divmod(elapsed_time.total_seconds(), 3600)[0])
    print("Instâncias: ", len(df_fault_final))

    if (len(df_fault_final) == 0):
        print("Falha " + str(ID) + " não existe!")
    else:
        labeled_data.loc[
            ((labeled_data["timestamp"] >= init_falha) & (labeled_data["timestamp"] <= end_falha)), 'rotulos_multi'] = ID

        labeled_data.loc[
            ((labeled_data["timestamp"] >= init_falha) & (labeled_data["timestamp"] <= end_falha)), 'rotulos_bin'] = 1

    print("Init falha: ", labeled_data.loc[labeled_data['rotulos_multi'] == ID]['timestamp'].min())
    print("End falha : ", labeled_data.loc[labeled_data['rotulos_multi'] == ID]['timestamp'].max())
    print("\nReal:")
    print("Init falha: ", init_falha)
    print("End falha : ", end_falha)

In [ ]:
Counter(labeled_data['rotulos_multi'])

In [ ]:
Counter(labeled_data['rotulos_bin'])

# Divisão dos bancos de dados

In [ ]:
pd.read_csv(data_folder + "banco_labeling_params-v0.csv", sep=';')[60:]

In [ ]:
x_treino = labeled_data.loc[:1212560,:]
x_treino = x_treino[x_treino['rotulos_multi'] == 0].copy()
y_treino = x_treino[['rotulos_multi']].copy()
x_treino.drop(['rotulos_multi', 'rotulos_bin', 'timestamp'], axis=1, inplace=True)

x_test = labeled_data.loc[1212560:,:]
y_test = labeled_data[['rotulos_multi']].loc[1212560:]

print("TREINO:")
print(x_treino.shape)
print(y_treino.shape)
print(Counter(y_treino['rotulos_multi']))
print()

print("\nTESTE:")
print(x_test.shape)
print(y_test.shape)
print(Counter(y_test['rotulos_multi']))
print()

# PCA 

#### Treinamento

In [ ]:
# Instancia o objeto PCA
pca = PCA(a=0.9)

# Treina o modelo PCA
pca.fit(x_treino)

print("\nPCs: ", pca.a)

#### Teste

In [ ]:
df_test = x_test.drop(['timestamp', 'rotulos_multi', 'rotulos_bin'], axis=1)
pca.predict(df_test)

In [ ]:
print(f'\nT2: {(pca.T2>pca.T2_lim).sum()/pca.T2.shape[0]}')
print(f'Q:  {(pca.Q>pca.Q_lim).sum()/pca.Q.shape[0]}')

pca.plot_control_charts()
plt.suptitle(f'Banco de teste');

In [ ]:
pca_results = pd.DataFrame(columns=['T2', 'Q'])
pca_results['T2'] = pca.T2
pca_results['Q'] = pca.Q 
pca_results['rotulos_multi'] = y_test.values

In [ ]:
pca_results['T2_classific'] = np.where(pca_results['T2'] > pca.T2_lim, 1, 0)
pca_results['Q_classific'] = np.where(pca_results['Q'] > pca.Q_lim, 1, 0)
pca_results['rotulos_bin'] = np.where(pca_results['rotulos_multi'] >= 1, 1, 0)
pca_results = pca_results.reset_index().drop(['index'], axis=1)
pca_results

In [ ]:
# Desempenho por T2

new_status = [0,1]

df_cm = pd.DataFrame(confusion_matrix(pca_results['rotulos_bin'], pca_results['T2_classific']), \
                     index=[i for i in new_status], columns=[i for i in new_status])

# Linha para normalizar os dados
df_cm_norm = round((df_cm.astype('float')/df_cm.sum(axis=1).values.reshape(-1,1)), 3)

# Gráfico da matriz de confusão
plt.figure(figsize=(6,5), dpi=100)
plt.title("Confusion Matrix - T2", fontsize=10)
ax = sn.heatmap(df_cm_norm, annot=True, cmap='PuBu')
ax.set_xlabel("PREDICTED CLASSES", fontsize=8)
ax.set_ylabel("REAL CLASSES", fontsize=8)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

tn, fp, fn, tp = confusion_matrix(pca_results['rotulos_bin'], pca_results['T2_classific']).ravel()
        
recall = tp/(tp+fn)
precision = tp/(tp+fp)
f1_score = 2*recall*precision/(recall+precision)
f2_score = 3*recall*precision/((2*precision)+recall)
f05_score = 1.5*recall*precision/((0.5*precision)+recall)
accuracy = (tp+tn)/(tp+tn+fp+fn)
specificity = tn/(fp+tn)
fpr = fp/(fp+tn)

metricas_df = pd.DataFrame([precision, recall, f1_score, f05_score, f2_score, specificity, accuracy, fpr, tp, \
                            tn, fp, fn]).T
metricas_df.columns = ['Precision', 'Recall', 'F-score(a=1)', 'F-score(a=0.5)', 'F-score(a=2)', \
                       'Specificity', 'Accuracy', 'FPR(FAR)', 'TP', 'TN', 'FP', 'FN']

print("\nOverall Precision:       {:.2f}%".format((metricas_df['Precision'].values.item()*100)))
print("Overall Recall:          {:.2f}%".format((metricas_df['Recall'].values.item()*100)))
print("Overall F-score(a=1):    {:.2f}%".format((metricas_df['F-score(a=1)'].values.item()*100)))
print("Overall F-score(a=0.5):  {:.2f}%".format((metricas_df['F-score(a=0.5)'].values.item()*100)))
print("Overall F-score(a=2):    {:.2f}%".format((metricas_df['F-score(a=2)'].values.item()*100)))
print("Overall Specificity:     {:.2f}%".format((metricas_df['Specificity'].values.item()*100)))
print("Overall FPR(FAR):        {:.2f}%".format((metricas_df['FPR(FAR)'].values.item()*100)))
print("Overall Accuracy:        {:.2f}%".format((metricas_df['Accuracy'].values.item()*100)))

In [ ]:
# Desempenho por Q

new_status = [0,1]

df_cm = pd.DataFrame(confusion_matrix(pca_results['rotulos_bin'], pca_results['Q_classific']), \
                     index=[i for i in new_status], columns=[i for i in new_status])

# Linha para normalizar os dados
df_cm_norm = round((df_cm.astype('float')/df_cm.sum(axis=1).values.reshape(-1,1)), 3)

# Gráfico da matriz de confusão
plt.figure(figsize=(6,5), dpi=100)
plt.title("Confusion Matrix - Q", fontsize=10)
ax = sn.heatmap(df_cm_norm, annot=True, cmap='PuBu')
ax.set_xlabel("PREDICTED CLASSES", fontsize=8)
ax.set_ylabel("REAL CLASSES", fontsize=8)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

tn, fp, fn, tp = confusion_matrix(pca_results['rotulos_bin'], pca_results['Q_classific']).ravel()
        
recall = tp/(tp+fn)
precision = tp/(tp+fp)
f1_score = 2*recall*precision/(recall+precision)
f2_score = 3*recall*precision/((2*precision)+recall)
f05_score = 1.5*recall*precision/((0.5*precision)+recall)
accuracy = (tp+tn)/(tp+tn+fp+fn)
specificity = tn/(fp+tn)
fpr = fp/(fp+tn)

metricas_df = pd.DataFrame([precision, recall, f1_score, f05_score, f2_score, specificity, accuracy, fpr, tp, \
                            tn, fp, fn]).T
metricas_df.columns = ['Precision', 'Recall', 'F-score(a=1)', 'F-score(a=0.5)', 'F-score(a=2)', \
                       'Specificity', 'Accuracy', 'FPR(FAR)', 'TP', 'TN', 'FP', 'FN']

print("\nOverall Precision:       {:.2f}%".format((metricas_df['Precision'].values.item()*100)))
print("Overall Recall:          {:.2f}%".format((metricas_df['Recall'].values.item()*100)))
print("Overall F-score(a=1):    {:.2f}%".format((metricas_df['F-score(a=1)'].values.item()*100)))
print("Overall F-score(a=0.5):  {:.2f}%".format((metricas_df['F-score(a=0.5)'].values.item()*100)))
print("Overall F-score(a=2):    {:.2f}%".format((metricas_df['F-score(a=2)'].values.item()*100)))
print("Overall Specificity:     {:.2f}%".format((metricas_df['Specificity'].values.item()*100)))
print("Overall FPR(FAR):        {:.2f}%".format((metricas_df['FPR(FAR)'].values.item()*100)))
print("Overall Accuracy:        {:.2f}%".format((metricas_df['Accuracy'].values.item()*100)))